# Frequencies in PWV rariation and comparison to Merra2

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2025-03-03 :
- last update 2025-03-03

- Kernel @usdf **w_2025_50*
- Office emac : mamba_py311
- Home emac : base (conda)


**Goal** : Show correlation holo /Merra

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
from pyzdcf import pyzdcf

In [ ]:
# where are stored the figures
pathfigs = "figsPWVFrequenciesAndMerra2"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
pathdata = "dataPWVFrequenciesAndMerra2"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datapath_input = os.path.join(pathdata,"timecurves") 
datapath_output = os.path.join(pathdata,"results") 
if not os.path.exists(datapath_input):
    os.makedirs(datapath_input) 
if not os.path.exists(datapath_output):
    os.makedirs(datapath_output) 

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
from astropy.timeseries import LombScargle

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
YEAR = 365.25
MONTHS6 = YEAR/2.
MONTHS4 = YEAR/3.
QUARTER = YEAR/4. 
DAY = 1.
MONTH = YEAR/12.
WEEK = 7*DAY

In [ ]:
FIGXSIZE_1 = 14
FIGYSIZE_1 = 8

FIGXSIZE_0 = 14
FIGYSIZE_0 = 5

In [ ]:
def LombScargle_analysis(dates, values, ax ,mode = "logxlogy",title="LombScargle - Spectrum",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit",label="Lomb Scargle", legendout = True, datecut = 0 ):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]

    
    values_centered = values - np.mean(values)
    

    # Nombre de points
    N = len(dates)

    # sigma
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)
    
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    

    freqs, power = LombScargle(dates, values_centered).autopower()
   
    ax.plot(freqs, power,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    #ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)
        


## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

## Merra2

In [ ]:
filename_m2 = "../SpectroMerra2/MerradataMerged/Merge_inst1_2d_asm_Nx_M2I1NXASM-2021-2024.csv"

In [ ]:
df_m = pd.read_csv(filename_m2)

In [ ]:
df_m["mjd"] = Time(pd.to_datetime(df_m.time.values)).mjd

In [ ]:
df_m.head()

### Spectro Hologram data

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

In [ ]:
version_results = "v5"
legendtag = {"v1" : "old v3.1.0",
            "v2" : "v3.1.0-PWV<10mm",
            "v3" : "v3.1.0-PWV<15mm",
            "v4" : "Auxtel holo v3.1.0",
            "v5" : "Auxtel holo v3.1.0 09/22 - 10/24"}

In [ ]:
atmfilenamesdict = {"v1" : "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy",
                    "v2" : "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy",
                    "v3" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy",
                    "v4" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z_spectrfullextend.npy",
                    "v5" : "u_dagoret_auxtel_atmosphere_202209_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip_20241016T184601Z_spectrfullextended.npy"}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
list(df_spec.columns)

In [ ]:
df_spec['ex_mjd']

### Remove spectra with red filter

In [ ]:
df_spec['FILTER'].unique()

In [ ]:
FLAG_REMOVE_FILTERS = True
if FLAG_REMOVE_FILTERS:
    df_spec=df_spec[df_spec["FILTER"] == 'empty']
    df_spec.reset_index(inplace=True)  

### Define if a target is faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:10]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

#### UTC

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

### Compute relative time to Mid-night

In [ ]:
def GetTimeToMidNight(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(local_time.year,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
    local_time_midnight = datetime(local_time_new.year,local_time_new.month,local_time_new.day)
    dt_hour = (local_time_new -local_time_midnight).seconds/3600.

    # we took the previous night mid-night , must subtract 24H
    if dt_hour > 12.:
        dt_hour_new = (dt_hour - 24.)
    else:
        dt_hour_new = dt_hour
        
    return dt_hour_new

In [ ]:
df_spec["dt_midnight"] = df_spec.apply(GetTimeToMidNight,axis=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
df_spec["dt_midnight"].hist(bins=48,range=(-12,12),ax=ax,facecolor="blue") 
ax.set_xlabel("time relative to midnight (hour)")
ax.set_title("Observation time")

### Compute Date relative to January

In [ ]:
def GetDateToMidJanuary(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new)

In [ ]:
#df_spec["Time_january"] = df_spec.apply(GetDateToMidJanuary,axis=1)

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new),local_time.year 

In [ ]:
df_spec[["Time_january","Year"]] = df_spec.apply(GetDateToMidJanuaryAndYear,axis=1,result_type="expand")

In [ ]:
df_spec[["Time_january","Year"]]

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="PShP",ax=ax,marker=".",c="b",lw=0.5,label="Merra2",ms=1,legend=leg)   
df_spec.plot(x="Time",y="OUTPRESS",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("OUTPRESS")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Pressure (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


#ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pressure_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX)   

df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=1,ms=1,ls="-",label="Merra2",legend=leg)   
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo vs time (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pwv_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


## Apply Quality selection cuts

In [ ]:
def getSelectionCut(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.1) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) 
    return cut

In [ ]:
def getSelectionCutNoPolar(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.1) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] != "HD185975")
    return cut

In [ ]:
def getSelectionCutWithPolar(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.1) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
cut = getSelectionCut(df_spec) 
cut_nopolar = getSelectionCutNoPolar(df_spec) 
cut_nopolar_bright = getSelectionCutNoPolar(df_spec) & (~df_spec["isFaint"])
cut_nopolar_faint = getSelectionCutNoPolar(df_spec) & (df_spec["isFaint"])
cut_wthpolar = getSelectionCutWithPolar(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]
df_spec_np = df_spec[cut_nopolar] 
df_spec_np_b = df_spec[cut_nopolar_bright]
df_spec_np_f = df_spec[cut_nopolar_faint]
df_spec_wp = df_spec[cut_wthpolar]

In [ ]:
print("Total number of Spectra          : ",len(df_spec))
print("Number of selected Spectra       : ",len(df_spec_sel))
print("Number of selected Polars        : ",len(df_spec_wp))
print("Number of selected Non-Polars    : ",len(df_spec_np))
print("Number of selected Non-Polars Bright : ",len(df_spec_np_b))
print("Number of selected Non-Polars Faint  : ",len(df_spec_np_f))

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)
df_spec_np.reset_index(drop=True,inplace=True)
df_spec_wp.reset_index(drop=True,inplace=True) 
df_spec_np_b.reset_index(drop=True,inplace=True)
df_spec_np_f.reset_index(drop=True,inplace=True)

In [ ]:
#List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
print("Polar            :",len(df_spec_wp["TARGET"].unique()),"\t", df_spec_wp["TARGET"].unique()) 
print("Non Polar        :",len(df_spec_np["TARGET"].unique()),"\t" ,df_spec_np["TARGET"].unique())
print("Non Polar Bright :",len(df_spec_np_b["TARGET"].unique()),"\t" ,df_spec_np_b["TARGET"].unique())
print("Non Polar Faint  :",len(df_spec_np_f["TARGET"].unique()),"\t",df_spec_np_f["TARGET"].unique())

## Recompute night boundaries

In [ ]:
#dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")

fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=0.5,ms=1,label="Merra2",legend=leg)     

df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=1,ms=1,ls="-",label="Merra2",legend=leg)  
df_spec_sel.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)

#df_spec_sel[df_spec_sel.Year==2022].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="g",lw=0.0,grid=True,label="2022",legend=leg,ms=5,alpha=1.)
#df_spec_sel[df_spec_sel.Year==2023].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="b",lw=0.0,grid=True,label="2023",legend=leg,ms=5,alpha=1.)
#df_spec_sel[df_spec_sel.Year==2024].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label="2024",legend=leg,ms=5,alpha=1.)

ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo",fontweight="bold")
ax.legend()

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()
 
ax.set_ylim(0.,15.)
#ax.set_xlim(TMIN,TMAX)

figname =f"{pathfigs}/pwvholoM2_allpoints_allnights"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()



df_m.plot(x="mjd",y="TQV",ax=ax,marker=".",c="b",lw=1,ms=1,ls="-",label="Merra2",legend=leg)  
df_spec_sel.plot(x="ex_mjd",y="PWV [mm]_x",ax=ax,marker='o',c="r",lw=0.0,grid=True,label=tag,legend=leg)


ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date (since January)")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo",fontweight="bold")
ax.legend()

#for key, tt in dn.items():
#    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

#ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()
 
ax.set_ylim(0.,15.)
#ax.set_xlim(TMIN,TMAX)

figname =f"{pathfigs}/pwvholoM2_allpoints_allnights_mjd"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
all_selected_nights 

## LombScargle

In [ ]:
x = df_spec["ex_mjd"].values
y = df_spec["PWV [mm]_x"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(x,y, ax=ax ,mode= "logxliny",
                 title = "LombScargle : PWV residuals",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel=" ",
                 label="Merra2 PWV")
figname =f"{pathfigs}/pwv_finalres_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()